<a href="https://colab.research.google.com/github/LoganLan0/Sketchbook/blob/master/plenoctree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

该notebook创建自Logan Lan

在内存13.6G，GPU tesla T4 15GB 

2023.04.23测试通过

分配虚拟内存，以供cuda运行


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 13.6 gigabytes of available RAM



检查nvidia

In [ ]:
!nvidia-smi


Sun Apr 23 04:07:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

检查python版本

In [ ]:
!python -V

Python 3.9.16


clone 官方仓库

In [ ]:
!git clone https://github.com/sxyu/plenoctree.git

Cloning into 'plenoctree'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 203 (delta 48), reused 39 (delta 39), pack-reused 139
Receiving objects: 100% (203/203), 86.42 KiB | 4.11 MiB/s, done.
Resolving deltas: 100% (111/111), done.


【手动操作】
1. 在左侧文件管理中，打开plenoctree文件夹，
2. 建立data/NeRF这样的目录结构
3. 将https://drive.google.com/drive/folders/128yBriW1IG_3NJ5Rp7APSTZsJqdJdfc1 这里的文件夹nerf_synthetic（注意是文件夹，不是zip文件）。在自己的硬盘中创立一个快捷方式。
4. 点击上方挂载自己的google drive
5. 将nerf_synthetic从drive/mydrive里面拖到data/NeRF文件夹下
6. 至此，完成了基本的目录结构。

接下来需要创建代码运行的基本环境。需要借助conda的力量。
我们需要在这里安装一下conda



In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...


然后conda升级到最新版本。

In [ ]:
!conda update -n base -c conda-forge conda

Solving environment: | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boltons-23.0.0             |     pyhd8ed1ab_0         296 KB  conda-forge
    charset-normalizer-3.1.0   |     pyhd8ed1ab_0          44 KB  conda-forge
    conda-23.3.1               |   py39hf3d152e_0         933 KB  conda-forge
    cryptography-40.0.2        |   py39h079d5ae_0         1.4 MB  conda-forge
    icu-72.1                   |       hcb278e6_0        11.4 MB  conda-forge
    jsonpatch-1.32             |     pyhd8ed1ab_0          14 KB  conda-forge
    jsonpointer-2.0            |             py_0           9 KB  conda-forge
    libcurl-7.88.1             |       hdc1c0ab_1         350 KB  conda-forge
    libsqlite-3.40.0           |      

运行完上面的命令后，通过输出版本的方式，检查conda安装是否正常。

In [ ]:
!conda --version

conda 23.3.1


有了conda之后，就可以根据官方的readme进行环境的配置了。


In [ ]:
%cd plenoctree

/content/plenoctree


*【可以跳过此步骤】在创建环境步骤中如果失败，使用下面的语句进行环境的删除，然后就可以重建了，如果不需要，请跳过此步骤*

In [ ]:
!conda remove -n plenoctree --all

In [ ]:
!conda env create -f environment.yml

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [ ]:
!source activate plenoctree && conda env list

# conda environments:
#
base                     /usr/local
plenoctree            *  /usr/local/envs/plenoctree



In [ ]:
%%bash
source activate plenoctree
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


其他必要环境的安装：

In [ ]:
!source activate plenoctree
!conda install -c conda-forge ta-lib

In [ ]:
!pip install --upgrade setuptools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.6.3
    Uninstalling setuptools-65.6.3:
      Successfully uninstalled setuptools-65.6.3


In [ ]:
%%bash
source activate plenoctree
pip install svox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached svox-0.2.32.tar.gz (43 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for svox: filename=svox-0.2.32-cp38-cp38-linux_x86_64.whl size=4023838 sha256=d69edee598b7b0bcc1700b158b291c62523bc48e1f3653e29a72631fbc6cd581
  Stored in directory: /root/.cache/pip/wheels/d7/5d/8a/cd99c7ccc4eb1949e63b023b6ebdedba60f7ff0bb84f6fb67d
Successfully built svox


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [89]:
%%bash
source activate plenoctree
pip install jax==0.2.9 jaxlib==0.1.59 numpy==1.18 chex flax==0.3.3 -f https://storage.googleapis.com/jax-releases/jax_releases.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 23.2 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.1.57
    Uninstalling jaxlib-0.1.57:
      Successfully uninstalled jaxlib-0.1.57


【必选的可选步骤】有GPU时，可以运行下面的步骤。
补充说明：如果COLAB的内存只有13GB左右，那就需要运行下面的代码，不然运行后虽然环境正确，但是会因为内存不足被killed。
所以这个时候如果有个15G的GPU，万事大吉，继续运行下面的代码，完成cuda的加成！可以继续训练！


In [98]:
%%bash
source activate plenoctree
pip install --upgrade "jax[cuda]" jaxlib==0.3.0 flax==0.3.5 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  Using cached jax-0.4.8-py3-none-any.whl
  Using cached flax-0.3.5-py3-none-any.whl (193 kB)
  Using cached jax-0.4.7-py3-none-any.whl
  Using cached jax-0.4.6.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached jax-0.4.5.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached jax-0.4.4.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached jax-0.4.3.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached jax-0.4.2.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setu

至此，我们完成了所有的安装！
下一步骤，引入变量并开始训练椅子。
！重要！
先把配置文件nerf_sh里的blender.yaml的maxstep改小，原来是两百万步，训练时间会特别长的。
这里我就改到两千步，来快速走一下后续的流程。


In [106]:
%%bash

export DATA_ROOT=./data/NeRF/nerf_synthetic/
export CKPT_ROOT=./data/Plenoctree/checkpoints/syn_sh16/
export SCENE=chair
export CONFIG_FILE=nerf_sh/config/blender
source activate plenoctree
python -m nerf_sh.train \
    --train_dir $CKPT_ROOT/$SCENE/ \
    --config $CONFIG_FILE \
    --data_dir $DATA_ROOT/$SCENE/

* Load train data
 Load Blender ./data/NeRF/nerf_synthetic//chair/ split train
 Generating rays
* Load test data
 Load Blender ./data/NeRF/nerf_synthetic//chair/ split test
 Generating rays
* Load model
* Prefetch
 1000/2000: i_loss=0.0064, avg_loss=0.0117, weight_l2=4.02e-03, lr=5.00e-05, 3223 rays/sec
 2000/2000: i_loss=0.0068, avg_loss=0.0076, weight_l2=4.02e-03, lr=5.00e-06, 3607 rays/sec


I0423 07:02:51.337743 140133292259136 xla_bridge.py:247] Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
I0423 07:02:51.421140 140133292259136 xla_bridge.py:247] Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
/usr/local/envs/plenoctree/lib/python3.8/site-packages/jax/_src/lib/xla_bridge.py:416: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  warnings.warn(
100%|██████████| 100/100 [00:04<00:00, 24.62it/s]
/usr/local/envs/plenoctree/lib/python3.8/site-packages/jax/_src/lib/xla_bridge.py:429: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
100%|██████████| 200/200 [00:07<00:00, 25.20it/s]
I0423 07:03:27.720456 140133292259136 checkpoints.py:249] Found no checkpoint files in data/Plenoctree/checkpoints/syn_sh16/chair
/u

上面的训练如果是2000步，将会执行大概十分钟左右的时间。
得到checkpoint文件后，就可以进入下一阶段了。
将nerf-sh转换成plenoctrees。

官方说是需要24GB GPU和15分钟左右。
这里colab 的15GB希望能运行吧。

1. 提取

In [108]:
%%bash

export DATA_ROOT=./data/NeRF/nerf_synthetic/
export CKPT_ROOT=./data/Plenoctree/checkpoints/syn_sh16
export SCENE=chair
export CONFIG_FILE=nerf_sh/config/blender
source activate plenoctree
python -m octree.extraction \
    --train_dir $CKPT_ROOT/$SCENE/ --is_jaxnerf_ckpt \
    --config $CONFIG_FILE \
    --data_dir $DATA_ROOT/$SCENE/ \
    --output $CKPT_ROOT/$SCENE/octrees/tree.npz

* Loading NeRF
* restore ckpt from ./data/Plenoctree/checkpoints/syn_sh16/chair/
Detected format: SH16
 Load Blender ./data/NeRF/nerf_synthetic//chair/ split train
data dim is 49
* Creating model
* Step 1: Grid eval
init grid torch.Size([134217728, 3])
* Calculating grid weights
torch.Size([4801556, 3]) tensor(-1.3506) tensor(1.4033)
 Building octree
svox.N3Tree(N=2, data_dim=49, depth_limit=8, capacity:872119/872119, data_format:SH16)
* Step 2: AA 8
svox.N3Tree(N=2, data_dim=49, depth_limit=8, capacity:872119/872119, data_format:SH16)
* Saving ./data/Plenoctree/checkpoints/syn_sh16/chair/octrees/tree.npz
 Load Blender ./data/NeRF/nerf_synthetic//chair/ split test
* Evaluation (before fine tune)
Evaluating octree
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /usr/local/envs/plenoctree/lib/python3.8/site-packages/lpips/weights/v0.1/vgg.pth
Average PSNR 20.931830377578734 SSIM 0.8498945260047912 LPIPS 0.21263381011784077


I0423 07:21:40.488798 140192483219264 checkpoints.py:252] Restoring checkpoint from data/Plenoctree/checkpoints/syn_sh16/chair/checkpoint_2000
100%|██████████| 100/100 [00:03<00:00, 29.43it/s]
/usr/local/envs/plenoctree/lib/python3.8/site-packages/svox/svox.py:109: UserWarning: map_location has been renamed to device and may be removed
  warn('map_location has been renamed to device and may be removed')
100%|██████████| 200/200 [00:07<00:00, 28.43it/s]
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 189MB/s]
100%|██████████| 200/200 [00:49<00:00,  4.03it/s]


2.优化

In [ ]:
%%bash 

export DATA_ROOT=./data/NeRF/nerf_synthetic/
export CKPT_ROOT=./data/Plenoctree/checkpoints/syn_sh16
export SCENE=chair
export CONFIG_FILE=nerf_sh/config/blender

source activate plenoctree

python -m octree.optimization \
    --input $CKPT_ROOT/$SCENE/octrees/tree.npz \
    --config $CONFIG_FILE \
    --data_dir $DATA_ROOT/$SCENE/ \
    --output $CKPT_ROOT/$SCENE/octrees/tree_opt.npz

3. 网页渲染压缩

In [ ]:
%%bash 

export DATA_ROOT=./data/NeRF/nerf_synthetic/
export CKPT_ROOT=./data/Plenoctree/checkpoints/syn_sh16
export SCENE=chair
export CONFIG_FILE=nerf_sh/config/blender
source activate plenoctree

python -m octree.compression \
    $CKPT_ROOT/$SCENE/octrees/tree_opt.npz \
    --out_dir $CKPT_ROOT/$SCENE/ \
    --overwrite